::::
:::{thebe-button}
:::
::::

# Process tracks


In [ ]:
from dev.docs.nbs import init

paths = init()

from boilercv_pipeline.experiments.e230920_subcool import (
    GBC,
    OBJECTS,
    THERMAL_DATA,
    TRACKS,
)
from boilercv_pipeline.sets import get_dataset
from dev.docs.nbs import HIDE
from matplotlib.figure import Figure
from numpy import diff, gradient
from pandas import DataFrame, Series, read_hdf

from boilercv.dimensionless_params import (
    fourier,
    kinematic_viscosity,
    nusselt,
    reynolds,
    thermal_diffusivity,
)

S = Series
"""Pandas series."""
TIME = "2023-09-20T17:14:18"
"""Timestamp of the trial to be analyzed."""
FIGURES: list[Figure] = []
"""Notebook figures available for export."""

# Track tuning
YPX_SURFACE_THRESHOLD = 400
"""Vertical position of bubble centroids considered attached to the surface."""
YPX_DEPARTURE_THRESHOLD = 420
"""Vertical position of bubble centroids considered to have departed the surface."""
MINIMUM_LIFETIME = 0.010  # (s)
"""Minimum bubble lifetime to consider."""

# Physical parameters
LATENT_HEAT_OF_VAPORIZATION = 2.23e6  # J/kg
LIQUID_DENSITY = 960  # kg/m^3
LIQUID_DYNAMIC_VISCOSITY = 2.88e-4  # Pa-s
LIQUID_ISOBARIC_SPECIFIC_HEAT = 4213  # J/kg-K
LIQUID_THERMAL_CONDUCTIVITY = 0.676  # W/m-K
VAPOR_DENSITY = 0.804  # kg/m^3

HIDE

## Aligning bubble departures

Exclude bubbles that did not originate from the boiling surface, or that had already departed the surface at the time of recording. Consider a bubble to have departed the surface when its centroid crosses a departure threshold which is about one average bubble diameter above the boiling surface. Define the origin for time of departure for each bubble in this fashion. The resulting time history in **Figure&NonBreakingSpace;2** shows bubble depth, velocity, and diameter for the remainder of its visible lifetime.

Most bubbles rise and collapse at similar rates. Two bubbles rise slower than the rest, but seem to collapse at about the same rate as others.


In [ ]:
path_time = TIME.replace(":", "-")
frametime = diff(get_dataset(path_time, stage="filled")["video"].time.values).mean()
objects: DataFrame = read_hdf((OBJECTS / f"objects_{path_time}").with_suffix(".h5"))  # pyright: ignore[reportAssignmentType]
thermal_data = read_hdf(THERMAL_DATA)
subcooling = thermal_data.subcool[TIME]
minimum_frame_lifetime = int(MINIMUM_LIFETIME // frametime)
raw_tracks: DataFrame = read_hdf((TRACKS / f"tracks_{path_time}").with_suffix(".h5"))  # pyright: ignore[reportAssignmentType]


def get_init(ser: S) -> float:
    """Get initial value of a series."""
    return ser.head(minimum_frame_lifetime).mean()


long_lived_objects = raw_tracks.query(f"frame_lifetime > {minimum_frame_lifetime}")
departing_long_lived_objects = (
    # Find rows corresponding to stagnant or invalid bubbles
    long_lived_objects.groupby("bubble", **GBC)
    .apply(
        # Don't assign any other columns until invalid rows have been filtered out
        lambda df: df.assign(
            yinit_px=lambda df: df["y_px"].pipe(get_init),
            # Initial y position is close to the surface
            began=lambda df: df["yinit_px"] > YPX_SURFACE_THRESHOLD,
            # When the bubble gets far enough away from the surface
            departed=lambda df: df["y_px"] < YPX_DEPARTURE_THRESHOLD,
        )
    )
    # Filter out invalid rows and drop the columns used to determine validity
    .pipe(lambda df: df[df["began"] & df["departed"]])
    .drop(columns=["began", "departed"])
    # Groupby again after filtering out invalid rows
    .groupby("bubble", **GBC)
    # Now columns that depend on the initial row (*.iat[0]) can be assigned
    .apply(
        lambda df: df.assign(
            frame=lambda df: df["frame"] - df["frame"].pipe(get_init),
            time=lambda df: df["time"] - df["time"].pipe(get_init),
            frame_lifetime=lambda df: df["frame"].iat[-1] - df["frame"].pipe(get_init),
            lifetime=lambda df: df["frame_lifetime"] * frametime,
            y_init=lambda df: df["y"].pipe(get_init),
            x_init=lambda df: df["x"].pipe(get_init),
            init_diameter=lambda df: df["diameter"].pipe(get_init),
            dy_init=lambda df: df["dy"].pipe(get_init),
            dy_init_px=lambda df: df["dy"].pipe(get_init),
            max_diameter=lambda df: df["diameter"].max(),
            diameter_rate_of_change=lambda df: gradient(df["diameter"], df["time"]),
        )
    )
)
processed_tracks = departing_long_lived_objects.assign(**{
    "Re_b": lambda df: reynolds(
        velocity=abs(df["dy"]),
        characteristic_length=df["diameter"],
        kinematic_viscosity=kinematic_viscosity(
            density=LIQUID_DENSITY, dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY
        ),
    ),
    "Re_b0": (
        lambda df: df.loc[:, ["bubble", "Re_b"]]
        .groupby("bubble", **GBC)
        .transform(get_init)
    ),
    "Fo_0": fourier(
        initial_bubble_diameter=departing_long_lived_objects["init_diameter"],
        liquid_thermal_diffusivity=thermal_diffusivity(
            thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
            density=LIQUID_DENSITY,
            isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
        ),
        time=departing_long_lived_objects["time"],
    ),
    "Nu_c": nusselt(
        heat_transfer_coefficient=-(
            2
            * VAPOR_DENSITY
            * LATENT_HEAT_OF_VAPORIZATION
            / subcooling
            * departing_long_lived_objects["diameter_rate_of_change"]
        ),
        characteristic_length=departing_long_lived_objects["diameter"],
        thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
    ),
    "beta": (lambda df: df["diameter"] / df["init_diameter"]),
}).query("Fo_0 > 0")